In [132]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from wordcloud import WordCloud
import regex as re
from nltk import download
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from sklearn.model_selection import GridSearchCV
from pickle import dump

In [133]:
data = pd.read_csv("https://raw.githubusercontent.com/4GeeksAcademy/NLP-project-tutorial/main/url_spam.csv")
data

,url,is_spam
0,https://briefingday.us8.list-manage.com/unsubs...,True
1,https://www.hvper.com/,True
2,https://briefingday.com/m/v4n3i4f3,True
3,https://briefingday.com/n/20200618/m#commentform,False
4,https://briefingday.com/fan,True
...,...,...
2994,https://www.smartcitiesworld.net/news/news/dee...,False
2995,https://www.youtube.com/watch,True
2996,https://techcrunch.com/2019/07/04/an-optimisti...,False
2997,https://www.technologyreview.com/2019/12/20/13...,False


In [134]:
print(data["is_spam"].unique())

[ True False]


In [135]:

data["spam"] = data["is_spam"].apply(lambda x: 1 if str(x).strip().lower() == "true" else 0).astype(int)
data.head()

,url,is_spam,spam
0,https://briefingday.us8.list-manage.com/unsubs...,True,1
1,https://www.hvper.com/,True,1
2,https://briefingday.com/m/v4n3i4f3,True,1
3,https://briefingday.com/n/20200618/m#commentform,False,0
4,https://briefingday.com/fan,True,1


In [136]:
print(data.shape)
print(f"Spam: {len(data.loc[data.spam == 1])}")
print(f"No spam: {len(data.loc[data.spam == 0])}")

(2999, 3)
Spam: 696
No spam: 2303


In [137]:
data = data.drop_duplicates()
data = data.reset_index(inplace = False, drop = True)
data.shape

(2369, 3)

In [138]:
import regex as re

def preprocess_text(text):
    # Remove any character that is not a letter (a-z) or white space ( )
    text = re.sub(r'[^a-z ]', " ", text)
    
    # Remove white spaces
    text = re.sub(r'\s+[a-zA-Z]\s+', " ", text)
    text = re.sub(r'\^[a-zA-Z]\s+', " ", text)

    # Multiple white spaces into one
    text = re.sub(r'\s+', " ", text.lower())

    # Remove tags
    text = re.sub("&lt;/?.*?&gt;"," &lt;&gt; ", text)

    return text.split()

data["Message"] = data["url"].apply(preprocess_text)
data.head()

,url,is_spam,spam,Message
0,https://briefingday.us8.list-manage.com/unsubs...,True,1,"[https, briefingday, us, list, manage, com, un..."
1,https://www.hvper.com/,True,1,"[https, www, hvper, com]"
2,https://briefingday.com/m/v4n3i4f3,True,1,"[https, briefingday, com, v, i]"
3,https://briefingday.com/n/20200618/m#commentform,False,0,"[https, briefingday, com, m, commentform]"
4,https://briefingday.com/fan,True,1,"[https, briefingday, com, fan]"


In [139]:
#Alejandro Hohmann
#7:59 PM
# alternate
#from nltk.corpus import words
#Russell Coonley
#8:41 PM
#[lemmatizer.lemmatize(word) for word in words if word not in stop_words and len(word) < 3]  <-- 1 line instead of 3

In [140]:
from nltk import download
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
download("wordnet")
lemmatizer = WordNetLemmatizer()

download("stopwords")
stop_words = stopwords.words("english")

def lemmatize_text(words, lemmatizer = lemmatizer):
    tokens = [lemmatizer.lemmatize(word) for word in words]
    tokens = [word for word in tokens if word not in stop_words]
    tokens = [word for word in tokens if len(word) > 3]
    return tokens

data["message"] = data["message"].apply(lemmatize_text)
data.head()

[nltk_data] Downloading package wordnet to
[nltk_data]     /home/codespace/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /home/codespace/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


KeyError: 'message'

In [99]:
import matplotlib.pyplot as plt
from wordcloud import WordCloud

wordcloud = WordCloud(width = 800, height = 800, background_color = "black", max_words = 1000, min_font_size = 20, random_state = 42)\
    .generate(str(data["message"]))

fig = plt.figure(figsize = (8, 8), facecolor = None)
plt.imshow(wordcloud)
plt.axis("off")
plt.show()

KeyError: 'message'

In [86]:
from sklearn.feature_extraction.text import TfidfVectorizer

tokens_list = data["url"]
tokens_list = [" ".join(tokens) for tokens in tokens_list]

vectorizer = TfidfVectorizer(max_features = 5000, max_df = 0.8, min_df = 5)
X = vectorizer.fit_transform(tokens_list).toarray()
y = data["url"]

X[:5]

ValueError: empty vocabulary; perhaps the documents only contain stop words

In [63]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42)

NameError: name 'X' is not defined